In [1]:
# .\.venv\Scripts\activate
# $env:Path += ";E:\fluidsynth download\bin" - add to path temporarily
# & "E:\fluidsynth download\bin\fluidsynth.exe" --version


print("python")

python


In [2]:
from pydub import AudioSegment

audio = AudioSegment.from_mp3(r"E:\instrumental music\trial 3\song-pilla chao.mp3")
audio.export("song.wav", format="wav")
# audio.export("song.wav", format="wav", codec="pcm_s16le")



<_io.BufferedRandom name='song.wav'>

In [3]:
import spleeter.separator

separator = spleeter.separator.Separator('spleeter:2stems')
separator.separate_to_file('song.wav', 'output_path')


INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:tensorflow:Using config: {'_model_dir': 'pretrained_models\\2stems', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.7
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_

In [ ]:
# sess.graph._unsafe_unfinalize()
# import tensorflow as tf

# tf.reset_default_graph()
# tf.Graph().as_default()

In [4]:
import crepe
import numpy as np
import librosa

# Load audio file
audio, sr = librosa.load("output_path/song/vocals.wav", sr=16000)



In [5]:
import tensorflow as tf


with tf.Graph().as_default():
    time, frequency, confidence, activation = crepe.predict(audio, sr, viterbi=True)

e:\instrumental music\trial 3\.venv\lib\site-packages\keras\engine\training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [6]:
import librosa

# Beat tracking
tempo, beat_frames = librosa.beat.beat_track(y=audio, sr=sr)
beat_times = librosa.frames_to_time(beat_frames, sr=sr)


In [7]:
import mido

def pitch_to_midi(pitch):
    return int(69 + 12 * np.log2(pitch / 440.0)) if pitch > 0 else 0

midi_notes = [pitch_to_midi(f) for f in frequency]


In [8]:
midi = mido.MidiFile()
track = mido.MidiTrack()
midi.tracks.append(track)

# Check if both lists are of equal length
if len(midi_notes) != len(beat_times):
    min_len = min(len(midi_notes), len(beat_times))
    midi_notes = midi_notes[:min_len]
    beat_times = beat_times[:min_len]

# Loop through the minimum length to avoid index errors
for i in range(min(len(midi_notes), len(beat_times))):
    note = midi_notes[i]
    track.append(mido.Message('note_on', note=note, velocity=64, time=int(beat_times[i] * 480)))
    track.append(mido.Message('note_off', note=note, velocity=64, time=int(beat_times[i] * 480 + 480)))

midi.save('output.mid')


In [ ]:
import fluidsynth

fs = fluidsynth.Synth()
fs.start()
sfid = fs.sfload("piano.sf2")
fs.program_select(0, sfid, 0, 0)
fs.midi_file_to_audio("output.mid", "instrumental_output.wav")
fs.delete()


AttributeError: module 'fluidsynth' has no attribute 'Synth'

In [ ]:
pip install pyfluidsynth

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyfluidsynth

# Initialize the synthesizer
fs = pyfluidsynth.Synth()
fs.start(driver="dsound")  # Specify driver if needed, "dsound" or "coreaudio" on Mac

# Load the soundfont
sfid = fs.sfload("piano.sf2")  # Make sure 'piano.sf2' is in the working directory
fs.program_select(0, sfid, 0, 0)

# Render MIDI to audio
fs.midi_file_to_audio("output.mid", "instrumental_output.wav")

# Clean up
fs.delete()


ModuleNotFoundError: No module named 'pyfluidsynth'

In [ ]:
import subprocess

# Define paths to the soundfont and MIDI file
soundfont_path = "FluidR3_GM.sf2"  # Path to the .sf2 file for the instrument
midi_path = "output.mid"      # Path to your MIDI file
output_wav_path = "instrumental_output.wav"  # Output audio file path

# Run fluidsynth with the command-line interface to convert MIDI to WAV
try:
    subprocess.run([
        "fluidsynth", "-ni", soundfont_path, midi_path, "-F", output_wav_path, "-r", "44100"
    ], check=True)
    print(f"Successfully created {output_wav_path}")
except subprocess.CalledProcessError as e:
    print("Error in running FluidSynth:", e)



FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
import subprocess

# Define paths to the soundfont and MIDI file
soundfont_path = "E:\instrumental music\trial 3\FluidR3_GM.sf2"  # Ensure this file exists in your working directory
midi_path = "E:\instrumental music\trial 3\output.mid"            # Ensure this file exists in your working directory
output_wav_path = "instrumental_output.wav"  # Output audio file path

# Run fluidsynth with the command-line interface to convert MIDI to WAV
try:
    subprocess.run([
        "fluidsynth", "-ni", soundfont_path, midi_path, "-F", output_wav_path, "-r", "44100"
    ], check=True)
    print(f"Successfully created {output_wav_path}")
except subprocess.CalledProcessError as e:
    print("Error in running FluidSynth:", e)
except FileNotFoundError:
    print("FluidSynth executable not found. Please ensure it is installed and in your PATH.")


In [9]:
import subprocess

# Define paths to the soundfont and MIDI file
soundfont_path = r"E:\instrumental music\trial 3\Mell Flutes.sf2"  # Ensure this file exists in your working directory
midi_path = r"E:\instrumental music\trial 3\output.mid"            # Ensure this file exists in your working directory
output_wav_path = r"E:\instrumental music\trial 3\instrumental_output.wav" # Output audio file path
fluidsynth_executable = r'E:\fluidsynth download\bin\fluidsynth.exe'

# Run fluidsynth with the command-line interface to convert MIDI to WAV
try:
    subprocess.run([
    fluidsynth_executable, "-ni", soundfont_path, midi_path, "-F", output_wav_path, "-r", "44100"
], check=True)
    print(f"Successfully created {output_wav_path}")
except subprocess.CalledProcessError as e:
    print("Error in running FluidSynth:", e)
except FileNotFoundError:
    print("FluidSynth executable not found. Please ensure it is installed and in your PATH.")


Successfully created E:\instrumental music\trial 3\instrumental_output.wav


In [ ]:
final_audio = AudioSegment.from_wav("instrumental_output.wav")
final_audio.export("instrumental_output.mp3", format="mp3")


Exception in thread Thread-25:
Traceback (most recent call last):
  File "E:\python download\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "e:\instrumental music\trial 3\.venv\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "E:\python download\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "E:\python download\lib\subprocess.py", line 1366, in _readerthread
    buffer.append(fh.read())
MemoryError
